[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/misSarah/matrix-multiplication/blob/master/%20codecuda.ipynb)

In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-nwigl_da
  Running setup.py bdist_wheel for NVCCPlugin ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-zig_7n28/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [60]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <math.h>
#define TILE_WIDTH 2
#define BLOCK_SIZE 64
#define n 8192

 __global__ void gpu_matrix_mult  ( int * Md, int * Nd, int * Pd )
{ 
       // identifiant de thread à deux dimensions, comme la matrice
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    // Pvaleur sert au stockage de la valeur calculée par le thread
    float Pvaleur = 0;
    for (int i = 0; i < n; ++i)
    {
        float MdElement = Md[ty * n + i];
        float NdElement = Nd[i  * n + tx];
        Pvaleur        += MdElement * NdElement;
    }
    // écrit la valeur calculée dans la matrice de résultat
    // chaque thread ne peut écrire qu'une valeur !
    Pd[ty * n + tx] = Pvaleur;
} 


__global__ void gpu_matrix_transpose(int* mat_in, int* mat_out) 
{
    unsigned int idx = blockIdx.x * blockDim.x + threadIdx.x;
    unsigned int idy = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx < n && idy < n) 
    {
        unsigned int pos = idy * n + idx;
        unsigned int trans_pos = idx * n + idy;
        mat_out[trans_pos] = mat_in[pos];
    }
}
void cpu_matrix_mult(int *A, int *B, int *C) {
       int d;
    for(int i=0;i<n;i++)
        for (int j=0;j<n;j++){
          d=0;
          for (int k=0;k<n;k++)      
                d+=A[i*n+k]*B[i*n+k];
          C[i*n+j] = d;
        }
}
// shared
__global__ void
MatrixMulSh( int *Md , int *Nd , int *Pd )
{      

        //Taking shared array to break the MAtrix in Tile widht and fatch them in that array per ele
          __shared__ int Mds [TILE_WIDTH][TILE_WIDTH] ;

           __shared__ int Nds [TILE_WIDTH][TILE_WIDTH] ;

         // calculate thread id
          unsigned int col = TILE_WIDTH*blockIdx.x + threadIdx.x ;
          unsigned int row = TILE_WIDTH*blockIdx.y + threadIdx.y ;

        for (int m = 0 ; m<n/TILE_WIDTH ; m++ ) // m indicate number of phase
       {
            Mds[threadIdx.y][threadIdx.x] =  Md[row*n + (m*TILE_WIDTH + threadIdx.x)]  ;
            Nds[threadIdx.y][threadIdx.x] =  Nd[ ( m*TILE_WIDTH + threadIdx.y) * n + col] ;
         __syncthreads() ; // for syncronizeing the threads
         int a=0;
         // Do for tile
           for ( int k = 0; k<TILE_WIDTH ; k++ )
                         a+= Mds[threadIdx.x][k] * Nds[k][threadIdx.y] ;
           Pd[row*n + col]=a;
           __syncthreads() ; // for syncronizeing the threads

     }
}


int main(int argc, char const *argv[])
{  
    
     

    // allocate memory in host RAM, h_cc is used to store CPU result
    int *h_a, *h_b, *h_c, *h_e;
    cudaMallocHost((void **) &h_a, sizeof(int)*n*n);
    cudaMallocHost((void **) &h_b, sizeof(int)*n*n);
    cudaMallocHost((void **) &h_c, sizeof(int)*n*n);
    cudaMallocHost((void **) &h_e, sizeof(int)*n*n);

    // random initialize matrix A
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] = rand() % 1024; 
        }
    }

    // random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            h_b[i * n + j] = rand() % 1024;
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start to count execution time of GPU version
    cudaEventRecord(start, 0);
    // Allocate memory space on the device 
    int *d_a, *d_b, *d_c,*d_e;
    cudaMalloc((void **) &d_a, sizeof(int) *n*n);
    cudaMalloc((void **) &d_b, sizeof(int) *n*n);
    cudaMalloc((void **) &d_c, sizeof(int) *n*n);
    cudaMalloc((void **) &d_e, sizeof(int) *n*n);
    // copy matrix A and B from host to device memory
    cudaMemcpy(d_a, h_a, sizeof(int)*n*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*n*n, cudaMemcpyHostToDevice);

   
    // Launch kernel 
    unsigned int grid_rows = sqrt(BLOCK_SIZE);
    unsigned int grid_cols = n/ grid_rows;
    
    
    dim3 dimGrid(grid_cols, grid_cols,1);
    dim3 dimBlock(grid_rows, grid_rows,1);
        gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_b, d_c);    
    
    // Transefr results from device to host 
    cudaMemcpy(h_c, d_c, sizeof(int)*n*n, cudaMemcpyDeviceToHost); 
    cudaThreadSynchronize();
    // time counting terminate
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
   
    // compute time elapse on GPU computing
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication on GPU: %f ms.\n\n", gpu_elapsed_time_ms);

    // start the CPU version
    cudaEventRecord(start, 0);

    cpu_matrix_mult(h_a, h_b, h_e);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication  on CPU: %f ms./n/ n",  cpu_elapsed_time_ms);
    // roughly compute speedup
        printf("all results are correct!!!, speedup = %f\n", cpu_elapsed_time_ms / gpu_elapsed_time_ms);
       
    // copy matrix A and B from host to device memory
    cudaMemcpy(d_e, h_e, sizeof(int)*n*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*n*n, cudaMemcpyHostToDevice);
    cudaEventRecord(start, 0);
    gpu_matrix_transpose<<<dimGrid, dimBlock>>>(d_b, d_e);  
    // Launch kernel 
    gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_e, d_c);    
    
    // Transefr results from device to host 
    cudaMemcpy(h_c, d_c, sizeof(int)*n*n, cudaMemcpyDeviceToHost); 
    cudaThreadSynchronize();
    // time counting terminate
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
   
    // compute time elapse on GPU computing
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed with transposed matrix on GPU: %f ms.\n\n", gpu_elapsed_time_ms);
  /*shared memory 
    */
        cudaMemcpy(d_a, h_a, sizeof(int)*n*n, cudaMemcpyHostToDevice);
        cudaMemcpy(d_b, h_b, sizeof(int)*n*n, cudaMemcpyHostToDevice);
     cudaMemcpy(d_c, h_c, sizeof(int)*n*n, cudaMemcpyHostToDevice);
    
    
  dim3 dimGrid1 ( n/TILE_WIDTH ,n/TILE_WIDTH ,1 ) ;

  dim3 dimBlock1( TILE_WIDTH, TILE_WIDTH, 1 ) ;
     cudaEventRecord(start, 0);
    // Launch kernel 
   MatrixMulSh<<<dimGrid1, dimBlock1>>>(d_a, d_e, d_c);    
    cudaEventRecord(stop, 0);
    // Transefr results from device to host 
    cudaMemcpy(h_c, d_c, sizeof(int)*n*n, cudaMemcpyDeviceToHost); 
    cudaThreadSynchronize();
    // time counting terminate
    
    cudaEventSynchronize(stop);
   
    // compute time elapse on GPU computing
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed with shared memory : %f ms.\n\n", gpu_elapsed_time_ms);
  

    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFreeHost(h_a);
    cudaFreeHost(h_b);
    cudaFreeHost(h_c);
    cudaFreeHost(h_e);
    return 0;
}


 





'Time elapsed on matrix multiplication on GPU: 34190.988281 ms.\n\nTime elapsed on matrix multiplication  on CPU: 2049948.125000 ms./n/ nall results are correct!!!, speedup = 59.955803\nTime elapsed with transposed matrix on GPU: 34231.457031 ms.\n\nTime elapsed with shared memory : 210242.468750 ms.\n\n'